## Set the paths

In [1]:
import yaml
from utils import get_face_utts

with open('configs/paths.yaml', 'r') as stream:
    PATHS = yaml.safe_load(stream)

with open(PATHS['ANNOTATIONS']['all'], 'r') as stream:
    datasets = yaml.safe_load(stream)

utts = get_face_utts(datasets, PATHS)


In [2]:
with open('./configs/train.yaml', 'r') as stream:
    train_config = yaml.safe_load(stream)

In [3]:
train_config['dataset']['train']

{'name': 'MeldDataset',
 'args': {'choose_frames': 'random_consecutive', 'num_frames': 1},
 'loader': {'name': '<dataloader class name>',
  'args': {'<dataloader argument keyword>': '<argument value>'}}}

## Data generator

In [4]:
# x = torch.squeeze(images_batch)
# x = x.permute(0, 2, 3, 1)


# print(x.shape)

# %matplotlib inline
# from matplotlib import pyplot as plt
# from matplotlib import animation
# from IPython.display import HTML

# # np array with shape (frames, height, width, channels)
# video = x

# fig = plt.figure()
# im = plt.imshow(video[0,:,:,:])

# plt.close() # this is required to not display the generated image

# def init():
#     im.set_data(video[0,:,:,:])

# def animate(i):
#     im.set_data(video[i,:,:,:])
#     return im

# anim = animation.FuncAnimation(fig, animate, init_func=init, frames=video.shape[0],
#                                interval=50)
# HTML(anim.to_html5_video())

In [ ]:
import timm
from pprint import pprint
model_names = timm.list_models(pretrained=True)
pprint(model_names)

In [20]:
import torch
import os
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as tf

from torch.utils import data
from torch.utils.data import DataLoader
from torchvision import transforms as tvtf


if train_config['dataset']['train']['name'] == 'MeldDataset':
    from erc.datasets.meld import MeldDataset as Dataset
else:
    pass

ds_train_kwargs = {'list_vid_paths':[os.path.join(PATHS['FACE_VIDEOS']['train'], utt) + '.mp4' for utt in utts['train']],
            'labels':[datasets['train'][utt]['Emotion'].lower() for utt in utts['train']], 
            **train_config['dataset']['train']['args']}

ds_train = Dataset(**ds_train_kwargs)

ds_train_gen = torch.utils.data.DataLoader(ds_train, batch_size=4, shuffle=True)


class BaseTimmModel(nn.Module):
    """Some Information about BaseTimmModel"""

    def _init_(self, num_classes, name, freeze_backbone=False, pretrained=True):
        super()._init_()
        self.model = timm.create_model(name, pretrained=pretrained)
        self.model.head = nn.Linear(self.model.head.in_features, num_classes)

    def forward(self, x):
        x = self.model(x)
        return x    
    
        
    

In [ ]:
model = BaseTimmModel(num_classes=7, name='efficientnet_b0', pretrained=True)
num_epoch = 10
for epoch in range(num_epoch):
    for image_batch, label_batch in ds_train_gen:
    

In [ ]:
val_acc_history = []

best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0
num_epochs = 10

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # Set model to training mode
        else:
            model.eval()   # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in ds_train_gen[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                # Get model outputs and calculate loss
                # Special case for inception because in training it has an auxiliary output. In train
                #   mode we calculate the loss by summing the final output and the auxiliary output
                #   but in testing we only consider the final output.
                if is_inception and phase == 'train':
                    # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                    outputs, aux_outputs = model(inputs)
                    loss1 = criterion(outputs, labels)
                    loss2 = criterion(aux_outputs, labels)
                    loss = loss1 + 0.4*loss2
                else:
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                _, preds = torch.max(outputs, 1)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

        # deep copy the model
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
        if phase == 'val':
            val_acc_history.append(epoch_acc)

    print()

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
print('Best val Acc: {:4f}'.format(best_acc))

# load best model weights
model.load_state_dict(best_model_wts)